<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Fake SQL Database class
from typing import Tuple, Sequence, List, Any, Callable, Dict, Iterator
from collections import defaultdict

# A few type aliases we'll use later
Row = Dict[str, Any]
WhereClause = Callable[[Row], bool] # Predicate for a single row
HavingClause = Callable[[List[Row]], bool] # Predicate over multiple ros

In [5]:
# Constructor for class to create column names and types
class Table:
  def __init__(self, columns: List[str], types: List[type]) -> None:
    assert len(columns) == len(types), "# of columns must == # of types"
    self.columns = columns
    self.types = types
    self.rows: List[Row] = [] # no data yet

  # Helper method to get the type of a column
  def col2type(self, col: str) -> type:
    idx = self.columns.index(col) # Find the index of the column
    return self.types[idx]
  
  def insert(self, values: list) -> None:
    # Check for the right number of values
    if len(values) != len(self.types):
      raise ValueError(f"You need to provide {len(self.types)} values")

    # Check for the right types of values
    for value, typ3 in zip(values, self.types):
      if not isinstance(value, typ3) and value is not None:
        raise TypeError(f"Expected type {typ3} but got {value}")

    # Add the corresponding dict as a "row"
    self.rows.append(dict(zip(self.columns, values)))
  
  def __getitem__(self, idx: int) -> Row:
    return self.rows[idx]
  
  def __iter__(self) -> Iterator[Row]:
    return iter(self.rows)

  def __len__(self) -> int:
    return len(self.rows)

  def __repr__(self):
    """Pretty representation of the table: columns then rows"""
    rows = "\n".join(str(row) for row in self.rows)
    return f"{self.columns}\n{rows}"

  # Let's update
  def update(self,
           updates: Dict[str, Any],
           predicate: WhereClause = lambda row: True):
    # First make sure the updates have valid names and types
    for column, new_value in updates.items():
      if column not in self.columns:
        raise ValueError(f"invalid column: {column}")

      typ3 = self.col2type(column)
      if not isinstance(new_value, typ3) and new_value is not None:
        raise TypeError(f"expected type {typ3}, but got {new_value}")

    # Now update
    for row in self.rows:
      if predicate(row):
        for column, new_value in updates.items():
          row[column] = new_value

In [7]:
users = Table(['user_id', 'name', 'num_friends'], [int, str, int])
users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])
assert len(users) == 11
assert users[1]['name'] == 'Dunn'

assert users[1]['num_friends'] == 2
users.update({'num_friends' : 3},
             lambda row: row['user_id'] == 1) 

assert users[1]['num_friends'] == 3